In [1]:
from flow.envs.base import Env

In [2]:
class DensityAwareBottleneckEnv(Env):
    def __init__(self, env_params, sim_params, network, simulator='traci'):
        for p in ADDITIONAL_ENV_PARAMS.keys():
            if p not in env_params.additional_params:
                raise KeyError(
                    'Environment parameter "{}" not supplied'.format(p))

        super().__init__(env_params, sim_params, network, simulator)
        
    @property
    def action_space(self):
        """See class definition."""
        return Box(
            low=-float("inf"),
            high=float("inf"),
            shape=(1, ),
            dtype=np.float32)

    @property
    def observation_space(self):
        """See class definition."""
        return Box(
            low=-float("inf"),
            high=float("inf"),
            shape=(1, ),
            dtype=np.float32)
    
    def _apply_rl_actions(self, rl_actions):
        """See parent class.

        To be implemented by child classes.
        """
        pass

    def compute_reward(self, rl_actions, **kwargs):
        """Outflow rate over last ten seconds normalized to max of 1."""
        pass

    def get_state(self):
        """See class definition."""
        pass

In [3]:
from flow.core.params import SumoParams, EnvParams, InitialConfig, VehicleParams, InFlows, NetParams
from flow.controllers import ContinuousRouter, SimLaneChangeController
from flow.core.params import SumoCarFollowingParams, SumoLaneChangeParams

# Ours
from custom_network import CustomBottleneckNetwork

In [4]:
HORIZON = 5000

N_CPUS = 1
N_ROLLOUTS = N_CPUS * 4 # Per iteration

In [5]:
vehicles = VehicleParams()
vehicles.add(
    veh_id="human",
    lane_change_controller=(SimLaneChangeController, {}),
    routing_controller=(ContinuousRouter, {}),
    car_following_params=SumoCarFollowingParams(
        speed_mode="all_checks",
    ),
    lane_change_params=SumoLaneChangeParams(
        lane_change_mode=0,
    ),
    num_vehicles=1)
    

In [6]:
ADDITIONAL_NET_PARAMS = {
    "speed_limit": 20,
}

additional_env_params = {}
ADDITIONAL_ENV_PARAMS = {}
additional_net_params = ADDITIONAL_NET_PARAMS.copy()

In [7]:
inflow = InFlows()
inflow.add(
    veh_type="human",
    edge="edge0",
    vehs_per_hour= 1000,
    depart_lane="random",
    depart_speed=10)

inflow.add(
    veh_type="human",
    edge="edge1",
    vehs_per_hour= 1000,
    depart_lane="random",
    depart_speed=10)

In [8]:
flow_params = dict(
    exp_tag = "DensityAwareBottleneck",
    env_name=DensityAwareBottleneckEnv,
    network=CustomBottleneckNetwork,
    simulator='traci',
    sim = SumoParams(sim_step=0.1, 
                        render=True,
                        restart_instance=True),
    env=EnvParams(
        warmup_steps=8000,
        sims_per_step=1,
        horizon=HORIZON,
        additional_params=additional_env_params,
    ),
    net = NetParams(
    inflows=inflow,
    additional_params=additional_net_params),
    veh=vehicles,
    initial=InitialConfig(
        spacing="uniform",
        min_gap=5,
        lanes_distribution=float("inf"),
        #edges_distribution=["2", "3", "4", "5"],
    ),
)

In [ ]:
from flow.core.experiment import Experiment

exp = Experiment(flow_params)

# run 
_ = exp.run(1)